In [ ]:
import numpy as np
import pandas as pd
import folium
import os
from pathlib import Path
from PIL import Image

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import branca.colormap as cm

## Definisi Variabel Utama pada Pemetaan Kerentanan Bangunan berdasarkan Bernardini dkk (2010)

Karena pada studi kasus Bandung dengan skenario gempa M7 dari sesar Lembang, MMI yang dirasakan oleh bangunan di Bandung hanya 8 hingga 9, hanya didefinisikan probabilitas kerusakan dengan MMI 8 dan 9 dalam bentuk *dictionary* pada Python

In [ ]:
#Keterangan Jumlah
none = 0
nearly_none = 1.5/100
few = 9/100
many = 35/100
most = 74.5/100
nearly_all = 97/100
alls = 1

In [ ]:
#Dictionary Damage Class A-E, MMI 8
damage_A_8 = {
    '0' : none,
    '1' : 1/3*few,
    '2' : 2*few,
    '3' : many,
    '4' : many,
    '5' : few
}

damage_B_8 = {
    '0' : 1/3*few,
    '1' : 2*few,
    '2' : many,
    '3' : many,
    '4' : few,
    '5' : none
}

damage_C_8 = {
    '0' : 7/3*few,
    '1' : many,
    '2' : many,
    '3' : few,
    '4' : none,
    '5' : none
}

damage_D_8 = {
    '0' : many + 7/3*few,
    '1' : many,
    '2' : few,
    '3' : none,
    '4' : none,
    '5' : none
}

damage_E_8 = {
    '0' : alls-few,
    '1' : few,
    '2' : none,
    '3' : none,
    '4' : none,
    '5' : none
}

In [ ]:
#Dictionary Damage Class A-E, MMI 9
damage_A_9 = {
    '0' : none,
    '1' : none,
    '2' : 1/3*few,
    '3' : 3*few,
    '4' : many,
    '5' : many
}

damage_B_9 = {
    '0' : none,
    '1' : 1/3*few,
    '2' : 2*few,
    '3' : many,
    '4' : many,
    '5' : few
}

damage_C_9 = {
    '0' : 1/3*few,
    '1' : 2*few,
    '2' : many,
    '3' : many,
    '4' : few,
    '5' : none
}

damage_D_9 = {
    '0' : 7/3*few,
    '1' : many,
    '2' : many,
    '3' : few,
    '4' : none,
    '5' : none
}

damage_E_9 = {
    '0' : many + 7/3*few,
    '1' : many,
    '2' : few,
    '3' : none,
    '4' : none,
    '5' : none
}

## Processing extracted CSV data to Folium Map

In [ ]:
df = pd.read_csv('Mapping/Cianjur 271122.csv')
df.head()

In [ ]:
def addMarkers(maps,lat,long,label,color):
    folium.Circle([lat,long],radius = 60,
                 fill = True, tooltip = label,
                  color = color).add_to(maps)

In [ ]:
#Membuat objek peta
cianjur = folium.Map(location=[-6.836374836193813, 107.03588445887296],zoom_start=11, tiles = 'Stamen Terrain')

for i in range(len(df)):
    if df.iloc[i,2] == 'Confined Masonry':
        addMarkers(cianjur,df.iloc[i,3],df.iloc[i,4],
              df.iloc[i,2],'red')
    elif df.iloc[i,2] == 'RC Infilled Masonry':
        addMarkers(cianjur,df.iloc[i,3],df.iloc[i,4],
              df.iloc[i,2],'blue')
    elif df.iloc[i,2] == 'Timber Structure':
        addMarkers(cianjur,df.iloc[i,3],df.iloc[i,4],
              df.iloc[i,2],'green')
    elif df.iloc[i,2] == 'Unconfined Masonry':
        addMarkers(cianjur,df.iloc[i,3],df.iloc[i,4],
              df.iloc[i,2],'purple')

#Generate Map
cianjur.save('Cianjur Typology.html')

# Processing the Bandung Area Damage Level D0 - D5

Considering the nearest fault in Bandung (Lembang), with damage calculation from Bernardini et al (2010)

In [ ]:
df = pd.read_excel('Mapping/Bandung Damage Map_Random Typology.xlsx', index_col = 0)

In [ ]:
kelass = []

for i in df.Tipologi:
    if(i == 'Confined Masonry'):
        kelas =  np.random.choice(['B','C','D'], 1, p=[0.27, 0.64, 0.09])[0]
        kelass.append(kelas)
    elif(i == 'RC Infilled Masonry'):
        kelas =  np.random.choice(['B','C','D'], 1, p=[0.27, 0.64, 0.09])[0]
        kelass.append(kelas)
    elif(i == 'Timber Structure'):
        kelas =  np.random.choice(['B','C','D'], 1, p=[0.09, 0.64, 0.27])[0]
        kelass.append(kelas)
    elif(i == 'Unconfined Masonry'):
        kelas =  np.random.choice(['A','B','C'], 1, p=[0.09, 0.82, 0.09])[0]
        kelass.append(kelas)

In [ ]:
df['MMI'] = df['MMI'].round(0).astype(int)

In [ ]:
df['Vulnerability_Class'] = kelass
df['Vulnerability_MMI'] = df['Vulnerability_Class'] + df['MMI'].astype(str)

In [ ]:
d0s = []
d1s = []
d2s = []
d3s = []
d4s = []
d5s = []

for i in df['Vulnerability_MMI']:
    if(i == 'A8'):
        d0s.append(damage_A_8["0"])
        d1s.append(damage_A_8["1"])
        d2s.append(damage_A_8["2"])
        d3s.append(damage_A_8["3"])
        d4s.append(damage_A_8["4"])
        d5s.append(damage_A_8["5"])
    elif(i == 'A9'):
        d0s.append(damage_A_9["0"])
        d1s.append(damage_A_9["1"])
        d2s.append(damage_A_9["2"])
        d3s.append(damage_A_9["3"])
        d4s.append(damage_A_9["4"])
        d5s.append(damage_A_9["5"])
    elif(i == 'B8'):
        d0s.append(damage_B_8["0"])
        d1s.append(damage_B_8["1"])
        d2s.append(damage_B_8["2"])
        d3s.append(damage_B_8["3"])
        d4s.append(damage_B_8["4"])
        d5s.append(damage_B_8["5"])
    elif(i == 'B9'):
        d0s.append(damage_B_9["0"])
        d1s.append(damage_B_9["1"])
        d2s.append(damage_B_9["2"])
        d3s.append(damage_B_9["3"])
        d4s.append(damage_B_9["4"])
        d5s.append(damage_B_9["5"])
    elif(i == 'C8'):
        d0s.append(damage_C_8["0"])
        d1s.append(damage_C_8["1"])
        d2s.append(damage_C_8["2"])
        d3s.append(damage_C_8["3"])
        d4s.append(damage_C_8["4"])
        d5s.append(damage_C_8["5"])
    elif(i == 'C9'):
        d0s.append(damage_C_9["0"])
        d1s.append(damage_C_9["1"])
        d2s.append(damage_C_9["2"])
        d3s.append(damage_C_9["3"])
        d4s.append(damage_C_9["4"])
        d5s.append(damage_C_9["5"])
    elif(i == 'D8'):
        d0s.append(damage_D_8["0"])
        d1s.append(damage_D_8["1"])
        d2s.append(damage_D_8["2"])
        d3s.append(damage_D_8["3"])
        d4s.append(damage_D_8["4"])
        d5s.append(damage_D_8["5"])
    elif(i == 'D9'):
        d0s.append(damage_D_9["0"])
        d1s.append(damage_D_9["1"])
        d2s.append(damage_D_9["2"])
        d3s.append(damage_D_9["3"])
        d4s.append(damage_D_9["4"])
        d5s.append(damage_D_9["5"])
    elif(i == 'E8'):
        d0s.append(damage_E_8["0"])
        d1s.append(damage_E_8["1"])
        d2s.append(damage_E_8["2"])
        d3s.append(damage_E_8["3"])
        d4s.append(damage_E_8["4"])
        d5s.append(damage_E_8["5"])
    elif(i == 'E9'):
        d0s.append(damage_E_9["0"])
        d1s.append(damage_E_9["1"])
        d2s.append(damage_E_9["2"])
        d3s.append(damage_E_9["3"])
        d4s.append(damage_E_9["4"])
        d5s.append(damage_E_9["5"])

In [ ]:
df['D0'] = d0s
df['D1'] = d1s
df['D2'] = d2s
df['D3'] = d3s
df['D4'] = d4s
df['D5'] = d5s

In [ ]:
df.to_excel('Mapping/Bandung Damage Map_Random Typology Paper 2023.xlsx')

## Mapping Building Vulnerabilities & Damage in Bandung with specified damage levels from D0 to D5

In [ ]:
colormap_D0 = cm.LinearColormap(colors=['green','yellow','orange','red'], index=[0, 0.33, 0.66,1],vmin=0,vmax=1)
colormap_D1D5 = cm.LinearColormap(colors=['green','yellow','orange','red'], index=[0, 0.166, 0.33, 0.5],vmin=0,vmax=0.5)

In [ ]:
df = pd.read_excel('Mapping/Cianjur Damage Map_Random Typology nonrev.xlsx')

In [ ]:
cianjur = folium.Map(location=[-6.836374836193813, 107.03588445887296],zoom_start=11, tiles = 'Stamen Terrain')

lat = list(df.lat)
lon = list(df.lon)
damage = list(df['D5'])

In [ ]:
for loc, dmg in zip(zip(lat, lon), damage):
    folium.Circle(
        location=loc,
        radius=50,
        fill=True,
        color=colormap_D1D5(dmg),
        #fill_opacity=0.7
    ).add_to(cianjur)
    
cianjur.add_child(colormap_D1D5)

In [ ]:
cianjur.save('Mapping/Cianjur - No Revision/Cianjur_Damage_5.html')